# q_error distribution

In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import math


plt.rcParams['axes.linewidth']=3
plt.rcParams['font.size']=18
plt.rcParams['lines.linewidth']=3

In [ ]:
import pickle
import numpy as np
from collections import Counter


def drawing(dataset,version):
    result_addr="./lecarb/estimator/mine/tree_inference_result/valid_"+dataset+"_"+version+".pkl"
    with open(result_addr, 'rb') as f:
        [inference_result,inference_time] = pickle.load(f)

    addr="./lecarb/estimator/mine/learning_model_prediction/valid_"+dataset+"_"+version+".pkl"
    with open(addr, 'rb') as f:
        [prediction,label,data_length] = pickle.load(f)
        
       
    prediction=prediction.cpu().detach().numpy()
    count_prediction=[float(i) for i in prediction]

    prediction_values=sorted(Counter(count_prediction).keys(),key=lambda x:x,reverse=False)
    
    label=np.around(label*data_length)
    label=[i[0] for i in label]
    

    loss=[] 
    t=[]
    mean_qerror=[]
    for threshold in prediction_values:
        add_time=0
        q_error=[]
        for i in range(len(prediction)):
            if prediction[i]<=threshold:
                q_error.append(1)
                add_time+=inference_time[i]
            else:
                p=np.around(prediction[i]*data_length)[0]
                
                
                l=label[i]
                if p==0 and l==0:
                    q_error.append(1)
                elif p==0:
                    q_error.append(l)
                elif l==0:
                    q_error.append(p)
                else:
                    q_error.append(max(p/l,l/p))
        add_time=add_time*1000/10000
        
        loss.append(add_time+np.mean(q_error))
        t.append(add_time)
        mean_qerror.append(np.mean(q_error))
    print("best eta:",prediction_values[loss.index(min(loss))])
    
    threshold=prediction_values[loss.index(min(loss))]
    add_time=0
    q_error=[]
    turn_to_precise=0
    for i in range(len(prediction)):
        if prediction[i]<=threshold:
            turn_to_precise+=1
            q_error.append(1)
            add_time+=inference_time[i]
        else:
            p=np.around(prediction[i]*data_length)[0]
            l=label[i]
            if p==0 and l==0:
                q_error.append(1)
            elif p==0:
                q_error.append(l)
            elif l==0:
                q_error.append(p)
            else:
                q_error.append(max(p/l,l/p))
    add_time=add_time*1000/10000
    print("max:",np.max(q_error),'99th:',np.percentile(q_error,99),'95th:',np.percentile(q_error,95),'90th:',np.percentile(q_error,90),'75th:',np.percentile(q_error,75),'50th:',np.percentile(q_error,50),'25th:',np.percentile(q_error,25),'mean:',np.mean(q_error))
    print("average time:",add_time,"ms/query")
    print("turn_to_precise",turn_to_precise)
    
    plt.xlabel('eta')
    plt.ylabel('loss')
    plt.plot(prediction_values,loss)
    plt.show()
    
    plt.xlabel('eta')
    plt.ylabel('incremented time of using tree')
    plt.plot(prediction_values,t)
    plt.show()
    
    plt.xlabel('eta')
    plt.ylabel('mean q_error after using tree')
    plt.plot(prediction_values,mean_qerror)
    plt.show()
    
    return prediction_values,loss,t,mean_qerror

In [ ]:
import numpy as np
from scipy.optimize import curve_fit,minimize
import matplotlib.pyplot as mpl

from sympy import symbols, diff
from sympy.functions import exp

def func1_for_diff(a,x0,sigma):
    x=symbols('x')
    f=a*exp(-(x-x0)**2/(2*sigma**2))
    derivative_f = diff(f, x)
    return f,derivative_f

def func2_for_diff(a,x0,sigma):
    x=symbols('x')
    f=a*exp(-(x-x0)**2/(2*sigma**2))+1
    derivative_f = diff(f, x)
    return f,derivative_f

# Let's create a function to model and create data
def func1(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

def func2(x, a, x0, sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))+1

def exponen(x,a,b,lamb):
    if type(x)==type([]):
        x=np.array(x)
    return a-b*np.exp(-(lamb*x))

def exponen_for_diff(a,b,lamb):
    x=symbols('x')
    f=a-b*exp(-(lamb*x))
    derivative_f = diff(f, x)
    return f,derivative_f

def fit2(x,y,fit_type,y_name):
    plt.plot(x, y, c='k', label='data')
    plt.scatter(x, y)
    if fit_type==1:
        popt, pcov = curve_fit(exponen, x, y,maxfev=1000000)
        ym = exponen(x, popt[0], popt[1], popt[2])
    elif fit_type==2:
        popt, pcov = curve_fit(func2, x, y,maxfev=1000000)
        ym = func2(x, popt[0], popt[1], popt[2])
    else:
        print("wrong type")
        return
        
    #popt returns the best fit values for parameters of the given model (func)
    
    plt.xlabel('eta')
    plt.ylabel(y_name)
    plt.plot(x, ym, c='r', label='fit')
    plt.legend()
    plt.show()
    return popt

def exponent_func(a,b,lamb):
    f=lambda x:a-b*np.exp(-(lamb*x))
    return f

def func2_func(a,x0,sigma):
    f=lambda x:a*np.exp(-(x-x0)**2/(2*sigma**2))+1
    return f



In [ ]:
def fit_and_find_best_eta(prediction_values,t,mean_qerror):
    x=symbols('x')
    popt1=fit2(prediction_values,t,fit_type=1,y_name='time')
    popt2=fit2(prediction_values,mean_qerror,fit_type=2,y_name='mqe')
    f1,_=exponen_for_diff(popt1[0],popt1[1],popt1[2])
    print("a:",popt1[0])
    print('b:',popt1[1])
    print('lambda:',popt1[2])
    f2,_=func2_for_diff(popt2[0],popt2[1],popt2[2])
    print("c:",popt2[0])
    print('x0:',popt2[1])
    print('sigma:',popt2[2])

    f=f1+f2
    
    print("time",f1)
    print("mean q error",f2)
    print(f)
    print("-----------------")
    
    func_f1=exponent_func(popt1[0],popt1[1],popt1[2])
    func_f2=func2_func(popt2[0],popt2[1],popt2[2])
    func_f=lambda x:func_f1(x)+func_f2(x)
    result=minimize(func_f,x0=[0.5],method='SLSQP',bounds=[(0,1)])
    
    
    loss=[]
    for i in range(len(t)):
        loss.append(t[i]+mean_qerror[i])
    plt.scatter(prediction_values,loss,label='loss')
    
    a=[i/10000 for i in range(0,10000)]
    b=[f.subs(x,i) for i in a]
    plt.plot(a,b,color='r',label='fit')
    best_eta=result['x'][0]
    print("best eta:",best_eta)
    
    plt.xlabel('eta')
    plt.ylabel('loss')
    plt.scatter(best_eta,f.subs(x,best_eta),color='green')
    plt.legend()
    plt.show()
    return best_eta,f.subs(x,best_eta),a,b


def test_for_best_eta(dataset,version,best_eta):
    result_addr="./lecarb/estimator/mine/tree_inference_result/"+dataset+"_"+version+".pkl"
    with open(result_addr, 'rb') as f:
        [inference_result,inference_time] = pickle.load(f)

    addr="./lecarb/estimator/mine/learning_model_prediction/test_"+dataset+"_"+version+".pkl"
    with open(addr, 'rb') as f:
        [prediction,label,data_length] = pickle.load(f)
        
    label=np.around(label*data_length)
    label=[i[0] for i in label]
    
#     for i in range(100):
#         print(label[i],inference_result[i])
#     return
    
    prediction=prediction.cpu().detach().numpy()
    add_time=0
    q_error=[]
    turn_to_precise=0
    for i in range(len(prediction)):
        if prediction[i]<=best_eta:
            q_error.append(1)
            turn_to_precise+=1
            add_time+=inference_time[i]
        else:
            p=np.around(prediction[i]*data_length)[0]
            l=label[i]
            if p==0 and l==0:
                q_error.append(1)
            elif p==0:
                q_error.append(l)
            elif l==0:
                q_error.append(p)
            else:
                q_error.append(max(p/l,l/p))
    add_time=add_time*1000/10000
    print("max:",np.max(q_error),'99th:',np.percentile(q_error,99),'95th:',np.percentile(q_error,95),'90th:',np.percentile(q_error,90),'75th:',np.percentile(q_error,75),'50th:',np.percentile(q_error,50),'25th:',np.percentile(q_error,25),'mean:',np.mean(q_error))
    print("average incremental time:",add_time,"ms/query")
    print("turn_to_precise",turn_to_precise)
    


# eta-loss/time/mean q_error curve

t : time

mqe : mean q_error

$loss=t+mqe$



$f(\eta) = a-be^{-\lambda \eta}$

$g(\eta)=1+ae^{-\frac{(\eta-\eta_{0})^{2}}{2\sigma^{2}}}$

$loss(\eta)=f(\eta)+g(\eta)$

minimization algorithm: SLSQP(SequentialLeastSquaresProgramming)

# census13

In [ ]:
prediction_values1,loss1,t1,mean_qerror1=drawing('census13','original')

In [ ]:
best_eta1,best_loss1,fit_x1,fit_y1=fit_and_find_best_eta(prediction_values1,t1,mean_qerror1)
test_for_best_eta("census13",'original',best_eta1)

# forest10

In [ ]:
prediction_values2,loss2,t2,mean_qerror2=drawing('forest10','original')

In [ ]:
best_eta2,best_loss2,fit_x2,fit_y2=fit_and_find_best_eta(prediction_values2,t2,mean_qerror2)
test_for_best_eta("forest10",'original',best_eta2)

# power7

In [ ]:
prediction_values3,loss3,t3,mean_qerror3=drawing('power7','original')

In [ ]:
best_eta3,best_loss3,fit_x3,fit_y3=fit_and_find_best_eta(prediction_values3,t3,mean_qerror3)
test_for_best_eta("power7",'original',best_eta3)

# dmv11

In [ ]:
prediction_values4,loss4,t4,mean_qerror4=drawing('dmv11','original')

In [ ]:
best_eta4,best_loss4,fit_x4,fit_y4=fit_and_find_best_eta(prediction_values4,t4,mean_qerror4)
test_for_best_eta("dmv11",'original',best_eta4)